In [1]:
import numpy as np
import pandas as pd

# Visualization modules
import matplotlib.pyplot as plt
import seaborn as sns

# Helpers
import helpers
import wrangle
import model
import viz

In [7]:
df = wrangle.wrangle_data('white')
df, _, _ = wrangle.train_test_validate_split(df)

In [8]:
df.head()

,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,ph,sulphates,alcohol,quality,type
1327,6.8,0.26,0.33,1.5,0.047,44.0,167.0,0.99280,3.12,0.44,10.5,6,white
622,5.3,0.32,0.12,6.6,0.043,22.0,141.0,0.99370,3.36,0.60,10.4,6,white
4737,6.1,0.24,0.32,9.0,0.031,41.0,134.0,0.99234,3.25,0.26,12.3,7,white
3503,6.9,0.33,0.31,4.2,0.040,21.0,93.0,0.98960,3.18,0.48,13.4,7,white
4239,5.7,0.28,0.36,1.8,0.041,38.0,90.0,0.99002,3.27,0.98,11.9,7,white


# Target variations

In [13]:
raw = df.quality
df['is_good'] = raw > raw.quantile(.75)
df['is_bad'] = raw > raw.quantile(.25)

## Best features

In [5]:
feats = [
    'alcohol',
    'volatile_acidity',
    'citric_acid',
    'residual_sugar',
    'total_sulfur_dioxide',
    'hydronium',
    'density',
    'ions',
    ]